In [27]:
import spacy
import PyPDF2
import os
import csv23
import sys
import ollama # We will user local ollama api here


In [28]:
!pip3 install PyPDF2
!pip3 install ollama
!pip3 install spacy
!pip3 install csv23
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.8 MB 3.4 MB/s eta 0:00:01
You should consider u

In [29]:

nlp = spacy.load("en_core_web_sm")

In [30]:
def extract_paragraph(pdf_file_path):
    paragraphs = []
    with open(pdf_file_path, "rb") as f:
        pdf_reader = PyPDF2.PdfReader(f)
        pages = pdf_reader.pages
        for page_num in range(len(pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            doc = nlp(text)
            #print(doc)

            for paragraph in doc.sents:
                paragraph_text = paragraph.text
                #print(paragraph_text)
                paragraphs.append(paragraph_text)
        return paragraphs

### Generate Question for each paragraph using ollama api and llama2-7b-gguf model

In [31]:

def generate_questions(paragraphs):
    print("stareted generating questions")
    questions_answers = []
    print("No of Paragraphs: ",len(paragraphs))
    for paragraph in paragraphs:
        #print("Paragraph: ", paragraph)
        
        print("Length:", len(paragraph))
        if len(paragraph) >= 100:
            prompt = f"""
            You are an question assistant and your job is to generate questions based on given paragraphs.
            Do not include anything except generated question.
            Do not mention here is the question.
            Generate questions for the following paragraph:\n\n{paragraph}\n\nQuestion:"""
            
            response = ollama.generate(
                model="llama2:latest",  # You can change the engine to other variants if needed
                stream= False,
                prompt=prompt,
                format="json"
            )
            print("Generated question: {}".format(response.response))
            questions_answers.append({"paragraph": paragraph, "questions": response['response']})
    return questions_answers


In [32]:
def save_to_csv(data, filename):
    print("stareted saving text")
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv23.DictWriter(file, fieldnames=["paragraph", "questions"])
        writer.writeheader()
        writer.writerows(data)

In [33]:
def main(pdf_path, output_csv):
    print("stareted main")
    paragraphs = extract_paragraph(pdf_path)
    questions_answers = generate_questions(paragraphs)
    save_to_csv(questions_answers, output_csv)

In [34]:
if __name__ == "__main__":
    pdf_path = "DAP 2020 13 Apr 2022.pdf"
    output_csv = f"{pdf_path}.csv"
    main(pdf_path, output_csv)

stareted main
stareted generating questions
No of Paragraphs:  11246
Length: 2
Length: 63
Length: 56
Length: 811


KeyboardInterrupt: 